# Read and Update Analyzers in Your Resource

This notebook demo how to read an existing analyzer and update it with a changed version.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.

In [1]:
%pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK.


In [2]:
import logging
import json
import os
import sys
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
AZURE_AI_API_VERSION = os.getenv("AZURE_AI_API_VERSION", "2024-12-01-preview")

# Add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from python.content_understanding_client import AzureContentUnderstandingClient

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_ENDPOINT,
    api_version=AZURE_AI_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/analyzer_management", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.11.11 (Linux-6.5.0-1025-azure-x86_64-with-glibc2.36)'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 400
Response headers:
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'IMDS/150.870.65.1475'
    'x-ms-request-id': 'e596b862-b95e-4662-8408-a83f4c0b1632'
    'Date': 'Wed, 19 Feb 2025 01:03:02 GMT'
    'Content-Length': '88'
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Re

## List all analyzers created in your resource

After the analyzer is successfully created, we can use it to analyze our input files.

In [3]:
response = client.get_all_analyzers()
print(f"Number of analyzers in your resource: {len(response['value'])}")
print(f"Analyzer details: {json.dumps(response['value'], indent=2)}")

Number of analyzers in your resource: 12
Analyzer details: [
  {
    "analyzerId": "prebuilt-read",
    "description": "Extract content elements such as words, barcodes, and formulas from documents.",
    "config": {
      "returnDetails": true,
      "enableOcr": true,
      "enableLayout": false,
      "enableBarcode": false,
      "enableFormula": false
    },
    "warnings": [],
    "status": "undefined",
    "scenario": "document"
  },
  {
    "analyzerId": "prebuilt-layout",
    "description": "Extract various content and layout elements such as words, paragraphs, and tables from documents.",
    "config": {
      "returnDetails": true,
      "enableOcr": true,
      "enableLayout": true,
      "enableBarcode": false,
      "enableFormula": false
    },
    "warnings": [],
    "status": "undefined",
    "scenario": "document"
  },
  {
    "analyzerId": "ImageTextAnalysis",
    "description": "Example to extract text from images and summarizes it.",
    "tags": {
      "projectId"

## Get analyzer details with id

Remember the analyzer id when you create it. You can use the id to look up detail analyzer definitions afterwards.

In [4]:
ANALYZER_ID = "PostCallAnalytics"

result = client.get_analyzer_detail_by_id(ANALYZER_ID)
print(json.dumps(result, indent=2))

{
  "analyzerId": "PostCallAnalytics",
  "description": "Post Call Analytics Analyzer based on the default template",
  "tags": {
    "projectId": "16f1d030-0fac-4ce0-a92c-88bcd6eee3d8",
    "templateId": "postCallAnalytics-2024-12-01"
  },
  "createdAt": "2025-02-06T19:31:09Z",
  "lastModifiedAt": "2025-02-06T19:31:11Z",
  "config": {
    "returnDetails": false
  },
  "fieldSchema": {
    "fields": {
      "Summary": {
        "type": "string",
        "method": "generate",
        "description": "A one-paragraph summary"
      },
      "Topics": {
        "type": "array",
        "method": "generate",
        "description": "Top 5 topics mentioned",
        "items": {
          "type": "string",
          "method": "generate"
        }
      },
      "Companies": {
        "type": "array",
        "method": "generate",
        "description": "List of companies mentioned",
        "items": {
          "type": "string",
          "method": "generate"
        }
      },
      "People": 

## Write analyzer details into json file for editing
Write the analyzer config into a json file.

In [5]:
output_dir = Path("../CU-Demo-Assets/analyzers/downloaded")

output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / f"{ANALYZER_ID}.json"
with open(output_file, "w") as f:
    json.dump(result, f, indent=2)
f.close()
print(f"Analyzer details written to {output_file}")



Analyzer details written to ../CU-Demo-Assets/analyzers/downloaded/PostCallAnalytics.json


## Pause until you have made the updates in the downloaded analyzer json file

In [6]:
pause = input("Press Enter to continue...")

## Update analyzer with id

Update an analyzer from a template.

In [7]:
import uuid

ANALYZER_TEMPLATE = f"../CU-Demo-Assets/analyzers/downloaded/{ANALYZER_ID}.json"

client.delete_analyzer(ANALYZER_ID)
response = client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=ANALYZER_TEMPLATE)
result = client.poll_result(response)

print(json.dumps(result, indent=2))

INFO:python.content_understanding_client:Analyzer PostCallAnalytics deleted.
INFO:python.content_understanding_client:Analyzer PostCallAnalytics create request accepted.
INFO:python.content_understanding_client:Request 0d1a74c9-544e-4eb3-aed2-045afb4d991f in progress ...
INFO:python.content_understanding_client:Request result is ready after 2.22 seconds.


{
  "id": "0d1a74c9-544e-4eb3-aed2-045afb4d991f",
  "status": "Succeeded",
  "result": {
    "analyzerId": "PostCallAnalytics",
    "description": "Post Call Analytics Analyzer based on the default template",
    "tags": {
      "projectId": "16f1d030-0fac-4ce0-a92c-88bcd6eee3d8",
      "templateId": "postCallAnalytics-2024-12-01"
    },
    "createdAt": "2025-02-12T23:08:57Z",
    "lastModifiedAt": "2025-02-12T23:08:59Z",
    "config": {
      "returnDetails": false,
      "disableContentFiltering": false
    },
    "fieldSchema": {
      "fields": {
        "Summary": {
          "type": "string",
          "method": "generate",
          "description": "A one-paragraph summary"
        },
        "Topics": {
          "type": "array",
          "method": "generate",
          "description": "Top 5 topics mentioned",
          "items": {
            "type": "string",
            "method": "generate"
          }
        },
        "Companies": {
          "type": "array",
          "m